In [3]:
#----mporting Necessary Libraries----------
import os
import pandas as pd

In [13]:
#--Directories--------
RAW_DIR = r"C:\Users\DELL\Data Science\Deep Learning\NLP\Projects\financial-qa\data\raw_docs"

In [35]:
def extract_chunks_from_page(filename, page_number, text, chunk_size=300, overlap=50):
    words = text.strip().split()
    chunks = []
    for x in range(0, len(words), chunk_size - overlap):
        chunk_words = words[x:x+chunk_size]
        if len(chunk_words) < 30:
            continue
        chunk_text = " ".join(chunk_words)
        chunk_id = f"{filename}_p{page_number}_c{x}"
        chunks.append({
            "filename": filename,
            "page": page_number,
            "chunk_id": chunk_id,
            "text": chunk_text
        })
    return chunks
    



def extract_text_from_pdf(RAW_DIR):
    all_chunks = []
    for specific_file in os.listdir(RAW_DIR):
        filepath = os.path.join(RAW_DIR, specific_file)
        with pdfplumber.open(filepath) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                text = page.extract_text()
                if not text:
                    continue
                text = text.replace("\n", " ").strip()
                chunks = extract_chunks_from_page(filepath, page_number, text)
                all_chunks.extend(chunks)
    return all_chunks

chunks_data = extract_text_from_pdf(RAW_DIR)

Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P1' is an invalid float value


In [4]:
df = pd.DataFrame(chunks_data)

NameError: name 'chunks_data' is not defined

In [37]:
df.shape

(3311, 4)

In [38]:
df.head()

,filename,page,chunk_id,text
0,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,1,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,"CIN: L65920MH1994PLC080618 HDFC Bank Limited, ..."
1,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,1,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,by way of remote e-voting will not be able to ...
2,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,2,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,"resolutions as read. Now, without any further ..."
3,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,2,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,"present. With permission of the members, I cal..."
4,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,3,C:\Users\DELL\Data Science\Deep Learning\NLP\P...,"Friends, we would like to recall last year whe..."


In [1]:
from sentence_transformers import SentenceTransformer

In [14]:
import os

os.listdir(RAW_DIR)

['HDFC_AGM_Transcript_Aug2024.pdf',
 'HDFC_BaselIII_Pillar3_2018.pdf',
 'HDFC_BaselIII_Pillar3_2023.pdf',
 'HDFC_Integrated_Annual_Report_2022.pdf',
 'HDFC_Integrated_Annual_Report_2023.pdf',
 'HDFC_Integrated_Annual_Report_2024.pdf.pdf',
 'HDFC_Liquidity_Coverage_Ratio_2022.pdf',
 'HDFC_Product_Risk_Disclosure_Options.pdf']

In [5]:
df = pd.read_csv(r"C:\Users\DELL\Data Science\Deep Learning\NLP\Projects\financial-qa\data\extracted_chunks.csv")

In [6]:
df.head()

,filename,page,chunk_id,text
0,HDFC_AGM_Transcript_Aug2024.pdf,1,HDFC_AGM_Transcript_Aug2024.pdf_p1_c0,"CIN: L65920MH1994PLC080618 HDFC Bank Limited, ..."
1,HDFC_AGM_Transcript_Aug2024.pdf,1,HDFC_AGM_Transcript_Aug2024.pdf_p1_c250,by way of remote e-voting will not be able to ...
2,HDFC_AGM_Transcript_Aug2024.pdf,2,HDFC_AGM_Transcript_Aug2024.pdf_p2_c0,"resolutions as read. Now, without any further ..."
3,HDFC_AGM_Transcript_Aug2024.pdf,2,HDFC_AGM_Transcript_Aug2024.pdf_p2_c250,"present. With permission of the members, I cal..."
4,HDFC_AGM_Transcript_Aug2024.pdf,3,HDFC_AGM_Transcript_Aug2024.pdf_p3_c0,"Friends, we would like to recall last year whe..."


In [7]:
text = df["text"].tolist()

In [8]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [9]:
embedding = model.encode(
    sentences=text,
    batch_size=32,
    show_progress_bar=True
)

Batches:   0%|          | 0/104 [00:00<?, ?it/s]

In [10]:
import numpy as np

embedding_matrix = np.vstack(embedding)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity


query = "What was the net profit of year 2024?"
query_embedding = model.encode([query])

In [19]:
similarities = cosine_similarity(query_embedding, embedding_matrix)[0]
similarities

array([0.07963666, 0.08650772, 0.07412349, ..., 0.29384857, 0.08142707,
       0.10899535], shape=(3311,), dtype=float32)

In [20]:
top_indices = similarities.argsort()[::-1][:5]

In [22]:
top_indices

array([2651,  631,  641, 2918,  835])

In [27]:
results = df.iloc[top_indices].copy()

In [28]:
results

,filename,page,chunk_id,text
2651,HDFC_Integrated_Annual_Report_2024.pdf.pdf,287,HDFC_Integrated_Annual_Report_2024.pdf.pdf_p28...,STANDALONE PROFIT AND LOSS ACCOUNT For the yea...
631,HDFC_Integrated_Annual_Report_2022.pdf,218,HDFC_Integrated_Annual_Report_2022.pdf_p218_c0,Profit and Loss Account For the year ended Mar...
641,HDFC_Integrated_Annual_Report_2022.pdf,225,HDFC_Integrated_Annual_Report_2022.pdf_p225_c250,"in '000 Year ended Year ended March 31, 2022 M..."
2918,HDFC_Integrated_Annual_Report_2024.pdf.pdf,413,HDFC_Integrated_Annual_Report_2024.pdf.pdf_p41...,CONSOLIDATED PROFIT AND LOSS ACCOUNT For the y...
835,HDFC_Integrated_Annual_Report_2022.pdf,306,HDFC_Integrated_Annual_Report_2022.pdf_p306_c0,Consolidated Profit and Loss Account For the y...


In [30]:
results["similarities"] = similarities[top_indices]

In [31]:
results

,filename,page,chunk_id,text,similarities
2651,HDFC_Integrated_Annual_Report_2024.pdf.pdf,287,HDFC_Integrated_Annual_Report_2024.pdf.pdf_p28...,STANDALONE PROFIT AND LOSS ACCOUNT For the yea...,0.589862
631,HDFC_Integrated_Annual_Report_2022.pdf,218,HDFC_Integrated_Annual_Report_2022.pdf_p218_c0,Profit and Loss Account For the year ended Mar...,0.571773
641,HDFC_Integrated_Annual_Report_2022.pdf,225,HDFC_Integrated_Annual_Report_2022.pdf_p225_c250,"in '000 Year ended Year ended March 31, 2022 M...",0.544387
2918,HDFC_Integrated_Annual_Report_2024.pdf.pdf,413,HDFC_Integrated_Annual_Report_2024.pdf.pdf_p41...,CONSOLIDATED PROFIT AND LOSS ACCOUNT For the y...,0.534386
835,HDFC_Integrated_Annual_Report_2022.pdf,306,HDFC_Integrated_Annual_Report_2022.pdf_p306_c0,Consolidated Profit and Loss Account For the y...,0.521615


In [37]:
df[["page"]][:5].to_dict()

{'page': {0: 1, 1: 1, 2: 2, 3: 2, 4: 3}}

In [38]:
embeddings = np.load(r"C:\Users\DELL\Data Science\Deep Learning\NLP\Projects\financial-qa\data\embedded_chunks.npy")

In [39]:
def get_top_k_similar_chunks(query, top_k=5):
    query_embedding = model.encode([query])
    similarity = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = similarity.argsort()[::-1][:top_k]
    results = df.iloc[top_indices].copy()
    results["Similarity"] = similarity[top_indices]
    return result

In [40]:
query = "What was the net profit of year 2024?"
result = get_top_k_similar_chunks(query)

In [41]:
result

,filename,page,chunk_id,text,Similarity
2651,HDFC_Integrated_Annual_Report_2024.pdf.pdf,287,HDFC_Integrated_Annual_Report_2024.pdf.pdf_p28...,STANDALONE PROFIT AND LOSS ACCOUNT For the yea...,0.589862
631,HDFC_Integrated_Annual_Report_2022.pdf,218,HDFC_Integrated_Annual_Report_2022.pdf_p218_c0,Profit and Loss Account For the year ended Mar...,0.571773
641,HDFC_Integrated_Annual_Report_2022.pdf,225,HDFC_Integrated_Annual_Report_2022.pdf_p225_c250,"in '000 Year ended Year ended March 31, 2022 M...",0.544387
2918,HDFC_Integrated_Annual_Report_2024.pdf.pdf,413,HDFC_Integrated_Annual_Report_2024.pdf.pdf_p41...,CONSOLIDATED PROFIT AND LOSS ACCOUNT For the y...,0.534386
835,HDFC_Integrated_Annual_Report_2022.pdf,306,HDFC_Integrated_Annual_Report_2022.pdf_p306_c0,Consolidated Profit and Loss Account For the y...,0.521615
